In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier # Import XGBoost
import numpy as np # Added for np.sum in fairness metrics
# from sklearn.utils.class_weight import compute_sample_weight # Not used in current reweighting logic


In [ ]:
def load_and_preprocess_data(file_path='fairness-dashboard/data/loan_data.csv'):
    """
    Loads the dataset and performs initial preprocessing steps:
    - Maps 'previous_loan_defaults_on_file' to numerical.
    - Creates 'age_group' from 'person_age' and drops original.
    - Keeps 'person_income' as numerical for normalization later.

    Returns:
        tuple: (X, y)
            X (pd.DataFrame): Features DataFrame before final scaling/encoding.
            y (pd.Series): Target Series.
    """
    try:
        df = pd.read_csv(file_path)
        print("Dataset loaded successfully.")
    except FileNotFoundError:
        print(f"Error: {file_path} not found. Please ensure the file is in the correct directory.")
        # Create a dummy DataFrame if the file is not found to allow the app to run
        data = {
            'person_gender': np.random.choice(['male', 'female'], 1000, p=[0.6, 0.4]),
            'person_education': np.random.choice(['Bachelor', 'Master', 'High School', 'Associate', 'Doctorate'], 1000),
            'person_income': np.random.randint(20000, 150000, 1000),
            'person_emp_exp': np.random.randint(0, 20, 1000),
            'person_home_ownership': np.random.choice(['RENT', 'OWN', 'MORTGAGE', 'OTHER'], 1000),
            'loan_amnt': np.random.randint(1000, 35000, 1000),
            'loan_intent': np.random.choice(['PERSONAL', 'EDUCATION', 'MEDICAL', 'VENTURE', 'HOMEIMPROVEMENT', 'DEBTCONSOLIDATION'], 1000),
            'loan_int_rate': np.random.uniform(5.0, 20.0, 1000),
            'loan_percent_income': np.random.uniform(0.05, 0.5, 1000),
            'cb_person_cred_hist_length': np.random.randint(2, 15, 1000),
            'credit_score': np.random.randint(500, 800, 1000),
            'previous_loan_defaults_on_file': np.random.choice([0, 1], 1000, p=[0.8, 0.2]),
            'loan_status': np.random.choice([0, 1], 1000, p=[0.7, 0.3]),
            'person_age': np.random.randint(20, 70, 1000)
        }
        df = pd.DataFrame(data)
        print("Using dummy data as 'loan_data.csv' was not found.")

    print("\n--- Initial Data Info ---")
    df.info()
    print("\n--- Missing Values Before Handling ---")
    print(df.isnull().sum())

    # 1. Map 'previous_loan_defaults_on_file'
    df['previous_loan_defaults_on_file'] = df['previous_loan_defaults_on_file'].map({'No': 0, 'Yes': 1})

    # 2. Convert 'person_age' to 'age_group' and drop original
    bins_age = [0, 18, 30, 45, 60, df['person_age'].max() + 1]
    labels_age = ['0-18', '18-30', '30-45', '45-60', '60 and above']
    df['age_group'] = pd.cut(df['person_age'], bins=bins_age, labels=labels_age, right=False, include_lowest=True)
    df.drop(columns=['person_age'], inplace=True)

    X = df.drop('loan_status', axis=1)
    y = df['loan_status']

    return X, y

# Execute data loading and initial preprocessing
X_raw, y = load_and_preprocess_data()

# Split data into training and testing sets (raw, before final scaling/encoding)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw, y, test_size=0.2, random_state=42, stratify=y)

# X_test_for_fairness is needed for fairness metrics, retaining original values
X_test_for_fairness = X_test_raw.copy()


In [ ]:
def preprocess_features(X_train_raw, X_test_raw):
    """
    Applies Min-Max Scaling to numerical features and One-Hot Encoding to categorical features.
    Handles missing values with median for numerical and mode for categorical.

    Args:
        X_train_raw (pd.DataFrame): Training features before final scaling/encoding.
        X_test_raw (pd.DataFrame): Testing features before final scaling/encoding.

    Returns:
        tuple: (X_train_processed, X_test_processed)
            X_train_processed (pd.DataFrame): Fully preprocessed training features.
            X_test_processed (pd.DataFrame): Fully preprocessed testing features.
    """
    numerical_features = X_train_raw.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features = X_train_raw.select_dtypes(include=['object', 'category']).columns.tolist()

    print(f"\nNumerical Features (for normalization): {numerical_features}")
    print(f"Categorical Features (for one-hot encoding): {categorical_features}")

    # Handle missing numerical values with median before scaling
    for col in numerical_features:
        median_val = X_train_raw[col].median()
        X_train_raw[col] = X_train_raw[col].fillna(median_val)
        X_test_raw[col] = X_test_raw[col].fillna(median_val)

    # Handle missing categorical values with mode before one-hot encoding
    for col in categorical_features:
        mode_val = X_train_raw[col].mode()[0]
        X_train_raw[col] = X_train_raw[col].fillna(mode_val)
        X_test_raw[col] = X_test_raw[col].fillna(mode_val)

    scaler = MinMaxScaler()
    onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

    X_train_scaled_num = scaler.fit_transform(X_train_raw[numerical_features])
    X_test_scaled_num = scaler.transform(X_test_raw[numerical_features])

    X_train_encoded_cat = onehot_encoder.fit_transform(X_train_raw[categorical_features])
    X_test_encoded_cat = onehot_encoder.transform(X_test_raw[categorical_features])

    X_train_scaled_num_df = pd.DataFrame(X_train_scaled_num, columns=numerical_features, index=X_train_raw.index)
    X_test_scaled_num_df = pd.DataFrame(X_test_scaled_num, columns=numerical_features, index=X_test_raw.index)

    encoded_feature_names = onehot_encoder.get_feature_names_out(categorical_features)
    X_train_encoded_cat_df = pd.DataFrame(X_train_encoded_cat, columns=encoded_feature_names, index=X_train_raw.index)
    X_test_encoded_cat_df = pd.DataFrame(X_test_encoded_cat, columns=encoded_feature_names, index=X_test_raw.index)

    X_train_processed = pd.concat([X_train_scaled_num_df, X_train_encoded_cat_df], axis=1)
    X_test_processed = pd.concat([X_test_scaled_num_df, X_test_encoded_cat_df], axis=1)

    print("\n--- Final X_train and X_test Info (after all preprocessing) ---")
    print("X_train info:")
    X_train_processed.info()
    print("\nX_test info:")
    X_test_processed.info()

    return X_train_processed, X_test_processed

# Execute feature preprocessing
X_train_processed, X_test_processed = preprocess_features(X_train_raw.copy(), X_test_raw.copy())


 Bias Mitigation - Reweighting
Reweighting is a pre-processing bias mitigation technique. It works by assigning different weights to individual data samples in the training set. The goal is to balance the representation of different protected groups (and their outcomes) in the training data, so that the model doesn't learn biases present in the original dataset.

The weight for each sample (g,y) (where g is the protected group and y is the outcome label) is calculated as:


w(g,y)= 
P(A=g)×P(Y=y∣A=g)
P(Y=y)
​
 

Where:

P(Y=y) is the overall proportion of instances with label y.

P(A=g) is the overall proportion of instances in protected group g.

P(Y=y∣A=g) is the proportion of instances with label y within group g.

This formula effectively up-weights under-represented groups/outcomes and down-weights over-represented ones, aiming for demographic parity (equal positive outcome rates) across groups.

In [6]:
def apply_reweighting(X_train_raw, y_train, protected_attribute_name, positive_outcome_label=1):
    """
    Calculates sample weights for reweighting bias mitigation, aiming for
    demographic parity (equal positive outcome rates) across groups of a
    single protected attribute.

    The weight for each sample (g, y) is calculated as:
    w(g, y) = P(Y=y) / (P(A=g) * P(Y=y | A=g))
    where:
    - P(Y=y) is the overall proportion of instances with label y.
    - P(A=g) is the overall proportion of instances in protected group g.
    - P(Y=y | A=g) is the proportion of instances with label y within group g.

    Args:
        X_train_raw (pd.DataFrame): The raw (unprocessed, but with age/income grouped)
                                    training features DataFrame.
        y_train (pd.Series): The training target labels.
        protected_attribute_name (str or list): The name(s) of the protected attribute column(s)
                                        (e.g., 'person_gender', ['person_gender', 'age_group']).
        positive_outcome_label (int): The label representing the positive outcome (e.g., 1 for approved).

    Returns:
        np.array: An array of sample weights, one for each sample in X_train_raw.
                  Returns None if the protected attribute is not found or has issues.
    """
    # Handle single vs. multiple protected attributes
    if isinstance(protected_attribute_name, list):
        # Create an intersectional group identifier
        if not all(col in X_train_raw.columns for col in protected_attribute_name):
            print(f"Error: One or more protected attributes {protected_attribute_name} not found in X_train_raw.")
            return None
        # Convert numerical attributes to binned categories for intersectional grouping
        temp_protected_attr_df = X_train_raw[protected_attribute_name].copy()
        for col in protected_attribute_name:
            if temp_protected_attr_df[col].dtype in ['int64', 'float64']:
                bins = pd.cut(temp_protected_attr_df[col], bins=5, right=False, include_lowest=True, duplicates='drop')
                temp_protected_attr_df[col] = bins
        prot_attr_series = temp_protected_attr_df.astype(str).agg('-'.join, axis=1)
    else:
        if protected_attribute_name not in X_train_raw.columns:
            print(f"Error: Protected attribute '{protected_attribute_name}' not found in X_train_raw.")
            return None
        prot_attr_series = X_train_raw[protected_attribute_name]
        if prot_attr_series.dtype in ['int64', 'float64']:
            bins = pd.cut(prot_attr_series, bins=5, right=False, include_lowest=True, duplicates='drop')
            prot_attr_series = bins

    print(f"\n--- Applying Reweighting for Protected Attribute(s): '{protected_attribute_name}' ---")

    # Combine X_train_raw and y_train for easier group-wise calculations
    train_data = X_train_raw.copy()
    train_data['target'] = y_train
    train_data['prot_attr_group'] = prot_attr_series.values # Align with train_data index

    # Calculate overall proportions
    p_y = train_data['target'].value_counts(normalize=True)

    # Calculate group proportions
    p_g = train_data['prot_attr_group'].value_counts(normalize=True)

    # Calculate conditional proportions P(Y=y | A=g)
    p_y_given_g = train_data.groupby('prot_attr_group')['target'].value_counts(normalize=True)

    sample_weights = np.zeros(len(train_data))

    # Iterate through each sample to assign a weight
    for i, row in train_data.iterrows():
        group = row['prot_attr_group'] # Get the group for the current sample
        label = row['target']

        p_y_val = p_y.get(label, 0.0)
        p_g_val = p_g.get(group, 0.0)
        p_y_given_g_val = p_y_given_g.get((group, label), 0.0)

        if p_g_val > 0 and p_y_given_g_val > 0:
            weight = p_y_val / (p_g_val * p_y_given_g_val)
        else:
            weight = 1.0 # Default to no reweighting if calculation is not possible for a specific instance

        sample_weights[train_data.index.get_loc(i)] = weight

    # Normalize weights to prevent them from overly influencing the loss function magnitude
    # This is a common practice to keep the sum of weights equal to the number of samples
    sample_weights = sample_weights / np.sum(sample_weights) * len(sample_weights)

    print(f"Reweighting applied. Sample weights calculated for '{protected_attribute_name}'.")
    print(f"  First 5 sample weights: {sample_weights[:5]}")
    print(f"  Sum of sample weights: {np.sum(sample_weights)}")
    print(f"  Min sample weight: {np.min(sample_weights)}, Max sample weight: {np.max(sample_weights)}")
    return sample_weights


 Model Training and Evaluation
This section defines a function to train and evaluate several common classification models. We will use:

Logistic Regression: A linear model for binary classification.

Decision Tree: A non-linear model that makes decisions based on feature values.

Random Forest: An ensemble method that builds multiple decision trees and merges their predictions.

XGBoost: A powerful gradient boosting framework known for its performance and speed.

Each model is trained on the preprocessed data, and its performance is assessed using:

Accuracy: The proportion of correctly classified instances.

Classification Report: Provides Precision, Recall, and F1-score for each class (0 and 1), and their averages.

Precision: The proportion of positive identifications that were actually correct.

Recall (Sensitivity): The proportion of actual positives that were identified correctly.

F1-score: The harmonic mean of Precision and Recall, providing a balance between the two.

In [7]:
def train_and_evaluate_models(X_train, y_train, X_test, y_test, models_dict, sample_weights=None):
    """
    Trains specified binary classification models and evaluates their performance.
    Can apply sample weights during training.

    Args:
        X_train (pd.DataFrame): Preprocessed training features.
        y_train (pd.Series): Training target.
        X_test (pd.DataFrame): Preprocessed testing features.
        y_test (pd.Series): Testing target.
        models_dict (dict): Dictionary of model names and scikit-learn model instances.
        sample_weights (np.array, optional): Array of sample weights for training.
                                            Defaults to None (no reweighting).

    Returns:
        dict: A dictionary containing results for each model, including accuracy,
              classification report, trained model, and predictions.
    """
    results = {}
    print("\n--- Model Training and Evaluation ---")
    for name, model in models_dict.items():
        print(f"\nTraining {name}...")
        try:
            # Directly attempt to fit with sample_weight if provided
            if sample_weights is not None:
                model.fit(X_train, y_train, sample_weight=sample_weights)
                print(f"  {name} trained WITH sample weights.")
            else:
                model.fit(X_train, y_train)
                print(f"  {name} trained WITHOUT sample weights (sample_weights was None).")
        except TypeError as e:
            # Fallback if the model genuinely does not support sample_weight
            print(f"  Warning: {name} does not support sample_weight. Training without it. Error: {e}")
            model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        # --- DIAGNOSTIC PRINT: Check predicted class distribution ---
        print(f"  {name} Predicted Class Distribution on Test Set:")
        print(pd.Series(y_pred).value_counts())
        print("-" * 40)
        # -----------------------------------------------------------

        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True)

        results[name] = {
            'accuracy': accuracy,
            'classification_report': report,
            'model': model,
            'y_pred': y_pred
        }
        print(f"{name} Accuracy: {accuracy:.4f}")
        print(f"{name} Classification Report:\n{classification_report(y_test, y_pred)}")

    print("\n--- Summary of Model Accuracies ---")
    for name, res in results.items():
        print(f"{name}: Accuracy = {res['accuracy']:.4f}")
    return results


Fairness Metrics Calculation
Beyond overall accuracy, it's crucial to assess if a model's performance is fair across different demographic groups. We calculate the following fairness metrics:

Proportion of Positive Predictions (PPR): The proportion of individuals in a specific group who received a positive prediction (e.g., loan approved).

True Positive Rate (TPR) / Recall: The proportion of actual positive cases within a group that were correctly identified as positive. This is also known as "Equal Opportunity."

From these, we derive two key fairness indicators:

Disparate Impact Ratio (DIR):


DIR= 
PPR 
privileged
​
 
PPR 
unprivileged
​
 
​
 

An ideal DIR is 1.0, meaning the positive prediction rate is equal for both groups. A value significantly below 1 (e.g., < 0.8) indicates that the unprivileged group is less likely to receive a positive outcome. A value significantly above 1 (e.g., > 1.25) indicates the unprivileged group is more likely to receive a positive outcome.

Equal Opportunity Difference (EOD):


EOD=TPR 
privileged
​
 −TPR 
unprivileged
​
 

An ideal EOD is 0.0, meaning the True Positive Rate is equal for both groups. This implies that the model is equally effective at identifying positive cases across different groups.

We define "privileged" and "unprivileged" groups based on their population size (majority vs. minority) for the purpose of calculating these ratios and differences.

In [8]:
def calculate_disparate_impact_ratio(group_metrics, privileged_group, unprivileged_group):
    """
    Calculates the Disparate Impact Ratio given group metrics and group labels.
    Returns the ratio and a message if calculation is not possible.
    """
    ppr_privileged = group_metrics.get(privileged_group, {}).get('PPR', 0.0)
    ppr_unprivileged = group_metrics.get(unprivileged_group, {}).get('PPR', 0.0)
    if ppr_privileged > 0:
        return ppr_unprivileged / ppr_privileged, None
    else:
        return None, "Disparate Impact Ratio: Cannot calculate (privileged group PPR is zero)."

def calculate_equal_opportunity_difference(group_metrics, privileged_group, unprivileged_group):
    """
    Calculates the Equal Opportunity Difference given group metrics and group labels.
    Returns the difference and a message if calculation is not possible.
    """
    tpr_privileged = group_metrics.get(privileged_group, {}).get('TPR', 0.0)
    tpr_unprivileged = group_metrics.get(unprivileged_group, {}).get('TPR', 0.0)
    if group_metrics.get(privileged_group, {}).get('Actual Positives', 0) > 0 and \
       group_metrics.get(unprivileged_group, {}).get('Actual Positives', 0) > 0:
        return tpr_privileged - tpr_unprivileged, None
    else:
        return None, "Equal Opportunity Difference: Not enough groups with actual positives to compare for the selected privileged/unprivileged groups."


def calculate_fairness_metrics(results, y_test, X_test_for_fairness, protected_attributes, positive_outcome_label=1, strategy_label='Standard'):
    """
    Calculates and prints Disparate Impact Ratio and Equal Opportunity Difference
    for each model across specified protected attributes, defining privileged/unprivileged
    based on majority/minority population.

    Args:
        results (dict): Dictionary containing model results (predictions, etc.).
        y_test (pd.Series): True labels for the test set.
        X_test_for_fairness (pd.DataFrame): Test features with original protected attributes.
        protected_attributes (list): List of column names to treat as protected attributes.
        positive_outcome_label (int/str): The label representing the positive outcome.
        strategy_label (str): Label indicating the mitigation strategy (e.g., 'Standard', 'Reweighted').
    Returns:
        pd.DataFrame: A DataFrame containing calculated fairness metrics for dashboard visualization.
    """
    print(f"\n--- Fairness Metrics Calculation for Strategy: {strategy_label} ---")
    fairness_data = [] # To store data for dashboard

    for model_name, model_results in results.items():
        # Add overall accuracy to fairness_data
        fairness_data.append({
            'Model': model_name,
            'Strategy': strategy_label,
            'Protected Attribute': 'Overall',
            'Group': 'Overall',
            'Metric Type': 'Accuracy',
            'Value': model_results['accuracy']
        })

        # Add macro-averaged Precision, Recall, and F1-Score
        macro_avg_report = model_results['classification_report']['macro avg']
        fairness_data.append({
            'Model': model_name,
            'Strategy': strategy_label,
            'Protected Attribute': 'Overall',
            'Group': 'Overall',
            'Metric Type': 'Precision (Macro Avg)',
            'Value': macro_avg_report['precision']
        })
        fairness_data.append({
            'Model': model_name,
            'Strategy': strategy_label,
            'Protected Attribute': 'Overall',
            'Group': 'Overall',
            'Metric Type': 'Recall (Macro Avg)',
            'Value': macro_avg_report['recall']
        })
        fairness_data.append({
            'Model': model_name,
            'Strategy': strategy_label,
            'Protected Attribute': 'Overall',
            'Group': 'Overall',
            'Metric Type': 'F1-Score (Macro Avg)',
            'Value': macro_avg_report['f1-score']
        })


        y_pred = model_results['y_pred']

        for attr in protected_attributes:
            if attr not in X_test_for_fairness.columns:
                print(f"Warning: Protected attribute '{attr}' not found in X_test_for_fairness. Skipping.")
                continue

            print(f"\n  Protected Attribute: {attr}")

            # Determine groups and their population sizes
            if X_test_for_fairness[attr].dtype in ['int64', 'float64']:
                temp_attr_series = pd.cut(X_test_for_fairness[attr], bins=5, labels=[f'{attr} Group {i+1}' for i in range(5)], right=False, include_lowest=True, duplicates='drop')
                group_populations = temp_attr_series.value_counts()
                groups = sorted(pd.Series(temp_attr_series.unique()).dropna().tolist())
            else:
                group_populations = X_test_for_fairness[attr].value_counts()
                groups = sorted(pd.Series(X_test_for_fairness[attr].unique()).dropna().tolist())

            if len(groups) < 2:
                print(f"    Only one group found for '{attr}'. Cannot calculate fairness metrics.")
                continue

            valid_groups = [g for g in groups if g in group_populations and group_populations[g] > 0]

            if len(valid_groups) < 2:
                print(f"    Not enough valid groups with samples for '{attr}'. Cannot calculate fairness metrics.")
                continue

            privileged_group_pop = group_populations[valid_groups].idxmax()
            unprivileged_group_pop = group_populations[valid_groups].idxmin()

            group_metrics = {}

            for group in groups:
                if X_test_for_fairness[attr].dtype in ['int64', 'float64']:
                    group_mask = (temp_attr_series == group)
                else:
                    group_mask = (X_test_for_fairness[attr] == group)

                y_true_group = y_test[group_mask]
                y_pred_group = y_pred[group_mask]

                if len(y_true_group) == 0:
                    print(f"    Group '{group}' has no samples in the test set. Skipping metric calculation for this group.")
                    continue

                tn, fp, fn, tp = confusion_matrix(y_true_group, y_pred_group, labels=[0, 1]).ravel()

                total_group_predictions = len(y_pred_group)
                ppr = tp + fp
                ppr_rate = ppr / total_group_predictions if total_group_predictions > 0 else 0

                actual_positives_in_group = tp + fn
                tpr = tp / actual_positives_in_group if actual_positives_in_group > 0 else 0

                group_metrics[group] = {
                    'PPR': ppr_rate,
                    'TPR': tpr,
                    'TP': tp,
                    'FP': fp,
                    'FN': fn,
                    'TN': tn,
                    'Total Samples': total_group_predictions,
                    'Actual Positives': actual_positives_in_group
                }
                print(f"    Group '{group}': PPR = {ppr_rate:.4f}, TPR = {tpr:.4f} (Population: {len(y_true_group)})")

                # Add individual group PPR and TPR to fairness_data
                fairness_data.append({
                    'Model': model_name,
                    'Strategy': strategy_label,
                    'Protected Attribute': attr,
                    'Group': str(group),
                    'Metric Type': 'PPR',
                    'Value': ppr_rate
                })
                fairness_data.append({
                    'Model': model_name,
                    'Strategy': strategy_label,
                    'Protected Attribute': attr,
                    'Group': str(group),
                    'Metric Type': 'TPR',
                    'Value': tpr
                })


            # Disparate Impact Ratio
            disparate_impact_ratio, dir_msg = calculate_disparate_impact_ratio(group_metrics, privileged_group_pop, unprivileged_group_pop)
            if disparate_impact_ratio is not None:
                print(f"    Disparate Impact Ratio (PPR_{unprivileged_group_pop} / PPR_{privileged_group_pop}): {disparate_impact_ratio:.4f}")
                fairness_data.append({
                    'Model': model_name,
                    'Strategy': strategy_label,
                    'Protected Attribute': attr,
                    'Group': f'DI ({unprivileged_group_pop} vs {privileged_group_pop})',
                    'Metric Type': 'Disparate Impact Ratio',
                    'Value': disparate_impact_ratio
                })
            else:
                print(f"    {dir_msg}")
                fairness_data.append({
                    'Model': model_name,
                    'Strategy': strategy_label,
                    'Protected Attribute': attr,
                    'Group': f'DI ({unprivileged_group_pop} vs {privileged_group_pop})',
                    'Metric Type': 'Disparate Impact Ratio',
                    'Value': np.nan
                })


            # Equal Opportunity Difference
            equal_opportunity_diff, eod_msg = calculate_equal_opportunity_difference(group_metrics, privileged_group_pop, unprivileged_group_pop)
            if equal_opportunity_diff is not None:
                print(f"    Equal Opportunity Difference (TPR_{privileged_group_pop} - TPR_{unprivileged_group_pop}): {equal_opportunity_diff:.4f}")
                fairness_data.append({
                    'Model': model_name,
                    'Strategy': strategy_label,
                    'Protected Attribute': attr,
                    'Group': f'EO ({privileged_group_pop} - {unprivileged_group_pop})',
                    'Metric Type': 'Equal Opportunity Difference',
                    'Value': equal_opportunity_diff
                })
            else:
                print(f"    {eod_msg}")
                fairness_data.append({
                    'Model': model_name,
                    'Strategy': strategy_label,
                    'Protected Attribute': attr,
                    'Group': f'EO ({privileged_group_pop} - {unprivileged_group_pop})',
                    'Metric Type': 'Equal Opportunity Difference',
                    'Value': np.nan
                })
    return pd.DataFrame(fairness_data)


 Main Execution Flow
This section orchestrates the entire pipeline:

Defines the machine learning models to be used.

Specifies the protected attributes for fairness analysis.

Executes the data preprocessing steps.

Trains and evaluates models without any bias mitigation (Standard approach).

Calculates fairness metrics for the standard models.

Applies the reweighting bias mitigation strategy and then trains and evaluates models with reweighting.

Calculates fairness metrics for the reweighted models.

Combines all fairness metrics into a single DataFrame and saves it as a Parquet file (combined_fairness_metrics.parquet). This file will be used by the interactive dashboard.

In [ ]:
def reweight_and_evaluate_models(
    X_train, y_train, X_test, y_test,
    models_dict,
    protected_attribute,
    X_train_for_fairness=None,
    verbose=True
):
    """
    Trains specified binary classification models with sample reweighting for bias mitigation.

    Args:
        X_train (pd.DataFrame): Preprocessed training features.
        y_train (pd.Series): Training target.
        X_test (pd.DataFrame): Preprocessed testing features.
        y_test (pd.Series): Testing target.
        models_dict (dict): Dictionary of model names and scikit-learn model instances.
        protected_attribute (str or list): Column name or list of column names in X_train (or X_train_for_fairness) to use for reweighting.
        X_train_for_fairness (pd.DataFrame, optional): DataFrame with original protected attributes (if X_train is encoded).
        verbose (bool): If True, prints accuracy and classification report.

    Returns:
        dict: A dictionary containing results for each model, including accuracy,
              classification report, trained model, and predictions.
    """
    # Use X_train_for_fairness if provided, else X_train
    if X_train_for_fairness is not None:
        if isinstance(protected_attribute, list):
            prot_attr = X_train_for_fairness[protected_attribute].astype(str).agg('-'.join, axis=1)
        else:
            prot_attr = X_train_for_fairness[protected_attribute]
    else:
        if isinstance(protected_attribute, list):
            prot_attr = X_train[protected_attribute].astype(str).agg('-'.join, axis=1)
        else:
            prot_attr = X_train[protected_attribute]

    # Ensure prot_attr is a pandas Series (important for .value_counts() and .map())
    if not isinstance(prot_attr, pd.Series):
        prot_attr = pd.Series(prot_attr)

    # Compute sample weights inversely proportional to group sizes
    group_counts = prot_attr.value_counts()
    
    # Create a Series of weights where index is the group value and value is 1.0 / count
    # Handle cases where a group count might be zero by setting weight to 0 (or a small number)
    weights_series = 1.0 / group_counts
    weights_series = weights_series.replace([np.inf, -np.inf], 0) # Replace inf/-inf (from 1/0) with 0

    # Map these weights back to the original prot_attr Series to get sample_weights for each instance
    sample_weights = prot_attr.map(weights_series)

    # Handle any NaN values that might occur if a group in prot_attr was not in group_counts
    # (e.g., if it was a NaN that got dropped by value_counts, or if it was a group with 0 count)
    # Defaulting to 1.0 means no reweighting for such instances.
    sample_weights = sample_weights.fillna(1.0)

    # Normalize weights to prevent them from overly influencing the loss function magnitude
    sample_weights = sample_weights / np.sum(sample_weights) * len(sample_weights)

    results = {}
    print("\n--- Model Training and Evaluation with Reweighting ---")
    for name, model in models_dict.items():
        print(f"\nTraining {name} with reweighting...")
        try:
            # Directly attempt to fit with sample_weight if provided
            if sample_weights is not None:
                model.fit(X_train, y_train, sample_weight=sample_weights)
                print(f"  {name} trained WITH sample weights.")
            else:
                model.fit(X_train, y_train)
                print(f"  {name} trained WITHOUT sample weights (sample_weights was None).")
        except TypeError as e:
            # Fallback if the model genuinely does not support sample_weight
            print(f"  Warning: {name} does not support sample_weight. Training without it. Error: {e}")
            model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True)

        results[name] = {
            'accuracy': accuracy,
            'classification_report': report,
            'model': model,
            'y_pred': y_pred,
            'sample_weights': sample_weights
        }
        if verbose:
            print(f"{name} Accuracy (reweighted): {accuracy:.4f}")
            print(f"{name} Classification Report (reweighted):\n{classification_report(y_test, y_pred)}")

    print("\n--- Summary of Model Accuracies (Reweighted) ---")
    for name, res in results.items():
        print(f"{name}: Accuracy (reweighted) = {res['accuracy']:.4f}")
    return results

# Define models to be used
selected_models = {
    'Logistic Regression': LogisticRegression(random_state=42, solver='liblinear'),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42)
}

# Define protected attributes for fairness analysis
protected_attributes_list = ['person_gender', 'age_group', 'person_education', 'person_home_ownership']

# 3. Train and evaluate models (standard)
print("\n================ Standard Model Training ================" )
model_results = train_and_evaluate_models(X_train_processed, y_train, X_test_processed, y_test, selected_models)
print("\n================ Fairness Metrics (Standard) ================" )
fairness_df_standard = calculate_fairness_metrics(model_results, y_test, X_test_for_fairness, protected_attributes_list, strategy_label='Standard')

# 4. Train and evaluate models with reweighting (intersectional example)
print("\n================ Reweighted Model Training (Intersectional: All Attributes) ================" )
reweighted_model_results_intersectional = reweight_and_evaluate_models(
    X_train_processed, y_train, X_test_processed, y_test,
    selected_models,
    protected_attribute=protected_attributes_list,
    X_train_for_fairness=X_train_raw
)
print("\n================ Fairness Metrics (Reweighted: Intersectional) ================" )
fairness_df_reweighted = calculate_fairness_metrics(reweighted_model_results_intersectional, y_test, X_test_for_fairness, protected_attributes_list, strategy_label='Reweighted')

# Combine dataframes for dashboard
combined_fairness_df = pd.concat([fairness_df_standard, fairness_df_reweighted], ignore_index=True)
print("\nCombined Fairness Data for Dashboard:")
print(combined_fairness_df.head())
print(combined_fairness_df.tail())

# Save the combined DataFrame to a Parquet file
output_file = 'combined_fairness_metrics.parquet'
combined_fairness_df.to_parquet(output_file, index=False)
print(f"\nCombined fairness metrics saved to {output_file}")